In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

In [18]:
template_block = blocks.create_block(template="Mention 'template block response' before answering: {query}")
generic_block = blocks.create_block()

template_resp = template_block("Say 'Hello this is an example'")
generic_resp = generic_block("Say 'Hello this is an example'")

print(f"{template_resp}")
print(f"{generic_resp}")

Template block response: Hello this is an example.
Hello this is an example.


In [25]:
template_block = blocks.create_block(stream=True, template="Respond in spanish: {query}")
generic_block = blocks.create_block(stream=True)

template_resp = template_block("Say hello to the user")
print("")
generic_resp = generic_block("Say hello to the user")

¡Hola usuario! ¿Cómo estás?
Hello! How can I assist you today?